In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

import numpy as np
from ipywidgets import interact
import folium

In [12]:
# loading data right from the source:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

# renaming the df column names to lowercase
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province_state': 'state', 'country_region': 'country'})
death_df = death_df.rename(columns={'province_state': 'state', 'country_region': 'country'})

In [16]:
confirmed_df

,uid,iso2,iso3,code3,fips,admin2,state,country,lat,long_,...,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,...,45,51,55,56,58,69,77,82,84,93
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,...,0,0,0,0,0,2,6,6,6,6
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.220800,-66.590100,...,64,79,100,127,174,239,286,316,316,452
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.335800,-64.896300,...,17,19,22,0,0,30,30,30,37,40
5,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6,6,6,6,6,7,8,10,12,12
6,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,5,5,10,15,18,19,20,24,28,29
7,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,0,0,0,0,0,0,0,0,1,2
8,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,0,0,0,0,2,3,3,4,4,4
9,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,2,4,5,5,5,5,5,6,9,10


In [14]:
confirmed_df.iloc[:,11:].sum(axis=1).sum()

2127690

In [15]:
death_df.iloc[:,12:].sum(axis=1).sum()

44429

In [4]:
class COVID_19:
    def __init__(self,confirmed_df,death_df):
        self.confirmed_df=confirmed_df
        self.death_df=death_df
        
    def pre_process(self):
        
        #calucating total cases tillnow
        self.confirmed_df['total_confirmed_tillnow']=self.confirmed_df.iloc[:,11:].sum(axis=1)
        self.death_df['total_death_tillnow']=self.death_df.iloc[:,12:].sum(axis=1)
        
        #group by state wise cases tillnow
        confirmed_df_state=self.confirmed_df[['state','total_confirmed_tillnow']].groupby(by='state').sum().reset_index()
        death_df_state=self.death_df[['state','total_death_tillnow']].groupby(by='state').sum().reset_index()
        
        #merge to make single df for U.S state
        df_state=pd.merge(confirmed_df_state,death_df_state,on=['state'],how='left')
        
        # groupby state wise to find day totals
        required_columns=list(self.confirmed_df.columns[11:-1])+['state']
        confirmed_df_daywise=self.confirmed_df[required_columns].groupby(by='state').sum().reset_index()
        required_columns=list(self.death_df.columns[12:-1])+['state']
        death_df_daywise=self.death_df[required_columns].groupby(by='state').sum().reset_index()
        
        return df_state,confirmed_df_daywise,death_df_daywise
    
    
    def topn_states_bubblechart(self,df,column,n):
        #filter topn by sorting
        topn=df.sort_values(column, ascending= False).head(n)
        #
        fig = px.scatter(topn, x="state", y=column, size=column, color="state",
                   hover_name="state", size_max=60)
        fig.update_layout(
        title='Top'+' '+str(n)+' '+column +' '+"states in U.S",
        xaxis_title="states",
        yaxis_title=str(column)+"Cases",
        autosize=False,
        width=1000,
        height=700
        )
       
        return fig.show()
    
    def lineplot_daywise(self,confirmed_df_daywise,death_df_daywise):
        
        #get x axis from dataframe
        x_data_confirm = np.array(confirmed_df_daywise.iloc[:,1:].columns)
        x_data_death= np.array(death_df_daywise.iloc[:,1:].columns)
        #get y-axis from datframe
        y_data_confirm = np.array(confirmed_df_daywise.iloc[:,1:].sum(axis=0))
        y_data_death = np.array(death_df_daywise.iloc[:,1:].sum(axis=0))
        
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x_data_confirm, y=y_data_confirm, mode='lines+markers',
                 name="Confimred_cases",
                #line=dict(color=colors[i], width=line_size[i]),
                connectgaps=True,
                #text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
                ));
        fig.add_trace(go.Scatter(x=x_data_death, y=y_data_death, mode='lines+markers',
                 name='Death_cases',
                #line=dict(color=colors[i], width=line_size[i]),
                connectgaps=True,
                #text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
                ));
        fig.update_layout(
                title="COVID 19 cases in U.S " ,
                xaxis_title='Date',
                yaxis_title='No. of Cases',
                margin=dict(l=20, r=20, t=40, b=20),
                paper_bgcolor="lightgrey",
                autosize=False,
                width=1000,
                height=700

            );

        fig.update_yaxes(type="linear")
        
        return fig.show()
    
    
    def US_map(self,state_dict,df,column):
        
        #code required for plotly
        df['code']=df['state'].map(state_dict)
        

        fig = go.Figure(data=go.Choropleth(
            locations=df['code'], # Spatial coordinates
            z = df[column].astype(float), # Data to be color-coded
            reversescale=True,
            locationmode = 'USA-states', # set of locations match entries in `locations`
            colorscale = 'blues',
            colorbar_title = "count",
        ))

        fig.update_layout(
            title_text = column+' '+' US COVID_19 by State',
            geo_scope='usa', # limite map scope to USA
            autosize=False,
            width=1000,
            height=700
        )

        return fig.show()
        
        

In [5]:
COVID_19=COVID_19(confirmed_df,death_df)
df_state,confirmed_df_daywise,death_df_daywise=COVID_19.pre_process()

In [7]:
df_state.total_confirmed_tillnow.sum()

2127690

In [8]:
df_state.total_death_tillnow.sum()

44429

# Topn Confirmed cases

In [178]:
COVID_19.topn_states_bubblechart(df_state,'total_confirmed_tillnow',20)

# Top n total deaths

In [180]:
COVID_19.topn_states_bubblechart(df_state,'total_death_tillnow',20)

# Confirmed cases and Death day wise

In [189]:
COVID_19.lineplot_daywise(confirmed_df_daywise,death_df_daywise)

# Confirmed case w.r.t State

In [191]:
COVID_19.US_map(state_dict,df_state,'total_confirmed_tillnow')

# Death case w.r.t State

In [190]:
COVID_19.US_map(state_dict,df_state,'total_death_tillnow')

In [173]:
state_dict={'Alabama': 'AL',
 'Alaska': 'AK',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin': 'WI',
 'Wyoming': 'WY'}

In [216]:
df_state['death_percent']=(df_state['total_death_tillnow']/df_state['total_confirmed_tillnow'])*100

In [217]:
df_state.head()

,state,total_confirmed_tillnow,total_death_tillnow,code,death_,death_percent
0,Alabama,11344,194,AL,1.710155,1.710155
1,Alaska,1304,27,AK,2.070552,2.070552
2,American Samoa,0,0,NaN,NaN,NaN
3,Arizona,14187,267,AZ,1.882005,1.882005
4,Arkansas,6395,83,AR,1.297889,1.297889


In [221]:
fig = px.bar(df_state, x='state', y='death_percent',
             hover_data=['state','total_confirmed_tillnow','total_death_tillnow'], color='death_percent',
             labels={'states':'U.S states '}, height=400)

fig.update_layout(
        title='Death out of confirmed',
        xaxis_title="states",
        yaxis_title="Death percent",
        autosize=False,
        width=1000,
        height=700
        )
fig.show()

In [208]:
fig = go.Figure()
fig.add_trace(go.Bar(
            x=df_state['total_confirmed_tillnow'],
            y=df_states['state'],
            orientation='h'))

fig.add_trace(go.Scatter(
            x=df_state['total_death_tillnow'],
            y=df_states['state'],
            mode='lines+markers',
            orientation='h'))
fig.update_layout(
        title='Top'+' '+str(5)+' '+'5' +' '+"states in U.S",
        xaxis_title="states",
        yaxis_title="Cases",
        autosize=False,
        width=1000,
        height=700
        )
#fig.update_layout(barmode='stack')
fig.show()